In [1]:
%run Classes_and_functions.ipynb

In [32]:
import hashlib
from sage.crypto.util import bin_to_ascii

In [20]:
hash_fun=hashlib.sha256

In [2]:
ea=216
eb=137
f=1

In [3]:
p=2^ea*3^eb*f-1

In [4]:
F=GF(p^2,'z')

In [5]:
E=MontgomeryEC(6,1,F)
show(E)

<__main__.MontgomeryEC instance at 0x12b4b9908>

In [6]:
while true:
    P=E.random_point()
    Pa=P*(3^eb*f)^2
    if Pa.order_torsion(2,ea)==2^ea:
        break
show(Pa)
while true:
    Q=E.random_point()
    Qa=Q*(3^eb*f)^2
    W=Pa.Weil_pairing(Qa,2^ea)
    if Qa.order_torsion(2,ea)==2^ea and check_order_F(W,2) == ea:
        break
show(Qa)

In [7]:
while true:
    P=E.random_point()
    Pb=P*(2^ea*f)^2
    if Pb.order_torsion(3,eb)==3^eb:
        break
show(Pb)
while true:
    Q=E.random_point()
    Qb=Q*(2^ea*f)^2
    W = Pb.Weil_pairing(Qb,3^eb)
    if Qb.order_torsion(3,eb)==3^eb and check_order_F(W,3) == eb:
        break
show(Qb)

In [87]:
DATABASE={}
PC={}
MESSAGES={}

In [89]:
'''
Create user
'''

name=raw_input("Write your username:")
if name in DATABASE.keys():
    print("Username already taken")
else:
    password=raw_input("Choose your password:")
    while True:
        RT=point_torsion(Pa,Qa,2,ea)
        R=RT[0]
        ta=RT[1]
        if (R*pow(2,ea-1)).x!=0:
            break
    phiA=iso_2_e(R,ea,E.a,E.b,Pb,Qb)
    EaA=phiA[0][0]
    EaB=phiA[0][1]
    phiAPb=phiA[1][0]
    phiAQb=phiA[1][1]

    DATABASE[name]=(EaA,EaB,phiAPb,phiAQb)
    cip_pass=hash_fun(password).hexdigest()
    PC[(name,cip_pass)]=(ta)
    print("Remember your password to acces to your private key!")

Write your username:Alice
Choose your password:1234
Remember your password to acces to your private key!


In [90]:
'''
Send message
'''
user=raw_input("Enter your username:")
password=raw_input("Enter your password:")
cip_pass=hash_fun(password).hexdigest()
if (user,cip_pass) not in PC.keys():
    print("Wrong user or password")
else:
    print("Who do you want to send the message to?")
    dest=raw_input()
    if dest not in DATABASE.keys():
        print("Username doesn't exist")
    else:
        print("Write your message:")
        m=raw_input()
        m=m.encode("hex")
        pk=DATABASE[dest]
        ST=point_torsion(Pb,Qb,3,eb)
        S=ST[0]
        tb=ST[1]
        phiB=iso_3_e(S,eb,E.a,E.b,Pa,Qa)
        EbA=phiB[0][0]
        EbB=phiB[0][1]
        phiBPa=phiB[1][0]
        phiBQa=phiB[1][1]
        phiAS1=pk[3]*tb
        phiAS=phiAS1+pk[2]
        psiB=iso_3_e(phiAS,eb,pk[0],pk[1])
        EbaA=psiB[0][0]
        EbaB=psiB[0][1]
        Eba=MontgomeryEC(EbaA,EbaB,F)
        j=str(Eba.j_invariant())
        h=hash_fun(j.encode('ascii')).hexdigest()
        c=ZZ(h,16)^^ZZ(m,16)
        if dest not in MESSAGES.keys():
            MESSAGES[(dest)]=((user,EbA,EbB,phiBPa,phiBQa,c))
        else:
            MESSAGES[(dest)]=(MESSAGES[dest],(user,EbA,EbB,phiBPa,phiBQa,c))

Enter your username:Bob
Enter your password:password
Who do you want to send the message to?
Alice
Write your message:
Hi, how are you?


In [91]:
'''
View messages
'''
user=raw_input("Enter your username:")
password=raw_input("Enter your password:")
cip_pass=hash_fun(password).hexdigest()
if (user,cip_pass) not in PC.keys():
    print("Wrong user or password")
elif user not in MESSAGES.keys():
    print("No new messages")
else:
    for i in range(len(MESSAGES[user])):
        pvk=PC[user,cip_pass]
        if len(MESSAGES[user][i])!=6:
            ct=MESSAGES[user]
            remitente=MESSAGES[user][0]
        else:
            ct=MESSAGES[user][i]
            remitente=MESSAGES[user][i][0]
        phiBR1=pvk*ct[4]
        phiBR=phiBR1+ct[3]
        psiA=iso_2_e(phiBR,ea,ct[1],ct[2])
        EbaA=psiA[0][0]
        EbaB=psiA[0][1]
        Eba=MontgomeryEC(EbaA,EbaB,F)
        j=str(Eba.j_invariant())
        h=hash_fun(j.encode('ascii')).hexdigest()
        m=ZZ(h,16)^^ZZ(ct[5],16)
        m=bin(m)[2:]
        while True:
            if len(m) % 8 !=0:
                m= '0'+m
            else:
                break

        print("New message from "+ remitente+":")
        print(bin_to_ascii(m))
        if len(MESSAGES[user][i])!=6:
            break
    del MESSAGES[user]

Enter your username:Alice
Enter your password:1234
New message from Bob:
Hi, how are you?
